# Статистика карточек товаров за период


Link: https://dev.wildberries.ru/openapi/analytics#tag/Voronka-prodazh/paths/~1api~1v2~1nm-report~1detail/post


Описание метода
Method: https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail

Метод формирует отчёт о товарах, сравнивая ключевые показатели — например, добавления в корзину, заказы и переходы в карточку товара — за текущий период с аналогичным прошлым.


Параметры brandNames,objectIDs, tagIDs, nmIDs могут быть пустыми [], тогда в ответе возвращаются все карточки продавца.


Если выбрано несколько параметров, в ответе будут карточки, в которых есть одновременно все эти параметры. Если карточки не подходят по параметрам запроса, вернётся пустой ответ [].


Можно получить отчёт максимум за последние 365 дней.


В данных предыдущего периода:

Данные в previousPeriod указаны за такой же период, что и в selectedPeriod.
Если дата начала previousPeriod раньше, чем год назад от текущей даты, она будет приведена к виду: previousPeriod.begin = текущая дата - 365 дней.

Максимум 3 запроса в минуту на один аккаунт продавца

Response Schema: application/json

- data	 / object
    - page	 / integer <int32> / Страница
    - isNextPage	 / boolean / Есть ли следующая страница (false - нет, true - есть)
    - cards	 / Array of objects / Array 
        - nmID	 / integer <int32> / Артикул WB
        - vendorCode	 / string / Артикул продавца
        - brandName	 / string / Название бренд
        - tags	 / Array of objects / Ярлыки / Array 
            - id	 / integer <int32> / ID ярлыка
            - name	 / string / Название ярлыка
        - object	 / object / Предмет
            - id	 / integer <int32> / ID предмета
            - name	/ string/ Название предмета/ 
        - statistics	/ object/ Статистика/ 
            - selectedPeriod	/ object/ Запрашиваемый период/ 
            - begin	/ string/ Начало периода
            - end	/ string/ Конец периода/ 
            - openCardCount	/ integer <int32>/ Количество переходов в карточку товара
            - addToCartCount	/ integer <int32>/ Положили в корзину, штук
            - ordersCount	/ integer <int32>/ Заказали товаров, шт
            - ordersSumRub	/ integer <int32>/ Заказали на сумму, руб.
            - buyoutsCount	/ integer <int32>/ Выкупили товаров, шт.
            - buyoutsSumRub	/ integer <int32>/ Выкупили на сумму, руб.
            - cancelCount	/ integer <int32>/ Отменили товаров, ш
            - cancelSumRub	/ integer <int32>/ Отменили на сумму, руб.
            - avgPriceRub	/ integer <int32>/ Средняя цена, руб.
            - avgOrdersCountPerDay	/ integer <int32>/ Среднее количество заказов в день, шт.
            - conversions	/ object/ Конверсии
            - addToCartPercent	/ integer <int32>/ Конверсия в корзину, % (Какой процент посетителей, открывших карточку товара, добавили товар в корзину)
            - artToOrderPercent	/ integer <int32>/ Конверсия в заказ, % (Какой процент посетителей, добавивших товар в корзину, сделали заказ)
            - buyoutsPercent	/ integer <int32>/ Процент выкупа, % (Какой процент посетителей, заказавших товар, его выкупили. Без учёта товаров, которые еще доставляются покупателю.)
            - previousPeriod	/ object/ Статистика за предыдущие 30 дней
            - periodComparison	/ object/ Сравнение двух периодов, в процентах
        - stocks	/ object/ Остатки/ 
            - stocksMp	 / integer <int32> / Остатки МП, шт. (Общее количество остатков на складе продавца)
            - stocksWb	 / integer <int32> / Остатки на складах WB (Общее количество остатков на складах WB)
- error	/ boolean/ Флаг ошибки
- errorText	/ string/ Описание ошибки
- additionalErrors / Array of object / Дополнительные ошибки

In [41]:
import requests
import json
import pandas as pd
from clickhouse_connect import get_client
from datetime import date, timedelta
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [42]:
# Retrieve API keys from environment variables
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv('KeySmart')
password = os.getenv('ClickHouse')


# Define headers for each project
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set

}


In [43]:
# Define the API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail"

# Automatically get yesterday's date
#yesterday_start = (date.today() - timedelta(days=1)).strftime('%Y-%m-%d 00:00:00')  # Start of yesterday
#yesterday_end = (date.today() - timedelta(days=1)).strftime('%Y-%m-%d 23:59:59')    # End of yesterday
yesterday_start = '2025-02-12 00:00:00'  # Start of yesterday
yesterday_end = '2025-02-12 23:59:59'   # End of yesterday


# Initialize variables
page = 1
def get_report(url, headers, begin, end, page, project_name ):
    all_data = []
    while True:
        # Define the request body
        request_body = {
            "period": {
                "begin": begin,  # Replace with the actual start date
                "end": end  # Replace with the actual end date
            },
            "orderBy": {
                "field": "ordersSumRub",  # Replace with the desired sorting field
                "mode": "desc"  # Replace with 'asc' for ascending or 'desc' for descending
            },
            "page": page  # Replace with the desired page number
        }

        # Convert the request body to JSON
        json_data = json.dumps(request_body)

        # Send the POST request
        response = requests.post(url, headers=headers, data=json_data)
        
        # Check the response status and content
        if response.status_code == 200:
            data = response.json()
            if data['data']['cards'] is None: # Stop if no more data is returned
                # Stop if no more data is returned or isNextPage is false
                break
            all_data.extend(data['data']['cards'])   # Add the data to the list
            print(f"Page {page} retrieved successfully for {project_name}.")
            page += 1  # Move to the next page
            time.sleep(20)
        else:
            print(f"Request failed with status code {response.status_code}.")
            print("Response text:", response.text)
            break
    return all_data

data_guten = get_report(url, headers_guten, yesterday_start, yesterday_end, page, 'WB-GutenTech')
data_giper = get_report(url, headers_giper, yesterday_start, yesterday_end, page, 'WB-ГиперМаркет')
data_kitchen = get_report(url, headers_kitchen, yesterday_start, yesterday_end, page, 'WB-KitchenAid')
data_smart = get_report(url, headers_smart, yesterday_start, yesterday_end, page, 'WB-Smart-Market')

Request failed with status code 429.
Response text: {
    "title": "too many requests",
    "detail": "Limited by global limiter, per seller 054b89ca-b532-5f6d-8b28-c54790d76a9b; See https://dev.wildberries.ru/openapi/api-information",
    "code": "461a0b83d6bd 2950e93b5fda",
    "requestId": "3c6f75779559c2e47ecf44b6609881ff",
    "origin": "s2sauth-ca",
    "status": 429,
    "statusText": "Too Many Requests",
    "timestamp": "2025-03-28T09:25:27Z"
}

Page 1 retrieved successfully for WB-ГиперМаркет.
Page 2 retrieved successfully for WB-ГиперМаркет.
Page 3 retrieved successfully for WB-ГиперМаркет.
Page 1 retrieved successfully for WB-KitchenAid.
Page 1 retrieved successfully for WB-Smart-Market.


In [44]:
# Function to flatten the JSON data for the current period
def flatten_json_current_period(cards):
    flattened_data = []
    for card in cards:
        nmID = card["nmID"]
        vendorCode = card["vendorCode"]
        brandName = card["brandName"]
        objectID = card["object"]["id"]
        objectName = card["object"]["name"]
        

        # Extract statistics for the selected period
        selected_period = card["statistics"]["selectedPeriod"]

        flattened_data.append({
            "nmID": nmID,
            "vendorCode": vendorCode,
            "brandName": brandName,
            "objectID": objectID,
            "objectName": objectName,
            "begin": selected_period["begin"],
            "end": selected_period["end"],
            "openCardCount": selected_period["openCardCount"],
            "addToCartCount": selected_period["addToCartCount"],
            "ordersCount": selected_period["ordersCount"],
            "ordersSumRub": selected_period["ordersSumRub"],
            "buyoutsCount": selected_period["buyoutsCount"],
            "buyoutsSumRub": selected_period["buyoutsSumRub"],
            "cancelCount": selected_period["cancelCount"],
            "cancelSumRub": selected_period["cancelSumRub"],
            "avgOrdersCountPerDay": selected_period["avgOrdersCountPerDay"],
            "avgPriceRub": selected_period["avgPriceRub"],
            "addToCartPercent": selected_period["conversions"]["addToCartPercent"],
            "cartToOrderPercent": selected_period["conversions"]["cartToOrderPercent"],
            "buyoutsPercent": selected_period["conversions"]["buyoutsPercent"],
            "stocksMp": card["stocks"]["stocksMp"],
            "stocksWb": card["stocks"]["stocksWb"]
        })

    return flattened_data

# Convert the flattened data to a DataFrame
flattened_data_guten = flatten_json_current_period(data_guten)
flattened_data_giper = flatten_json_current_period(data_giper)
flattened_data_kitchen = flatten_json_current_period(data_kitchen)
flattened_data_smart = flatten_json_current_period(data_smart)

df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)

# Add the 'Project' column to each DataFrame before concatenation
df_guten['Project'] = 'WB-GutenTech'
df_giper['Project'] = 'WB-ГиперМаркет'
df_kitchen['Project'] = 'WB-KitchenAid'
df_smart['Project'] = 'WB-Smart-Market'

# Combine all campaign data
combined_df = pd.concat([df_guten, df_giper, df_kitchen, df_smart], ignore_index=True)
combined_df['Marketplace'] = 'Wildberries'
print("Columns in combined_campaigns:", combined_df.columns.tolist())

# Display the DataFrame
combined_df

Columns in combined_campaigns: ['Project', 'nmID', 'vendorCode', 'brandName', 'objectID', 'objectName', 'begin', 'end', 'openCardCount', 'addToCartCount', 'ordersCount', 'ordersSumRub', 'buyoutsCount', 'buyoutsSumRub', 'cancelCount', 'cancelSumRub', 'avgOrdersCountPerDay', 'avgPriceRub', 'addToCartPercent', 'cartToOrderPercent', 'buyoutsPercent', 'stocksMp', 'stocksWb', 'Marketplace']


,Project,nmID,vendorCode,brandName,objectID,objectName,begin,end,openCardCount,addToCartCount,...,cancelCount,cancelSumRub,avgOrdersCountPerDay,avgPriceRub,addToCartPercent,cartToOrderPercent,buyoutsPercent,stocksMp,stocksWb,Marketplace
0,WB-ГиперМаркет,329624233.0,CM-00-00003493,CASO,3936.0,Су-вид,2025-02-12 00:00:00,2025-02-12 23:59:59,6.0,4.0,...,0.0,0.0,1.0,68950.0,67.0,25.0,100.0,3.0,2.0,Wildberries
1,WB-ГиперМаркет,292476664.0,8700216588713,GILLETTE,1527.0,Бритвы безопасные,2025-02-12 00:00:00,2025-02-12 23:59:59,22.0,22.0,...,8.0,18080.0,22.0,2260.0,100.0,100.0,64.0,320.0,0.0,Wildberries
2,WB-ГиперМаркет,232867811.0,Б0062793,Oral-B,557.0,Электрические зубные щетки,2025-02-12 00:00:00,2025-02-12 23:59:59,37.0,7.0,...,1.0,15669.0,2.0,16864.0,19.0,29.0,50.0,10.0,6.0,Wildberries
3,WB-ГиперМаркет,231259893.0,I01002,AND,636.0,Тонометры,2025-02-12 00:00:00,2025-02-12 23:59:59,490.0,72.0,...,2.0,4776.0,13.0,2388.0,15.0,18.0,83.0,2280.0,0.0,Wildberries
4,WB-ГиперМаркет,263732078.0,Б0067288,Braun,620.0,Бритвы электрические,2025-02-12 00:00:00,2025-02-12 23:59:59,30.0,2.0,...,0.0,0.0,1.0,29967.0,7.0,50.0,100.0,9.0,4.0,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620,WB-Smart-Market,288465481.0,8700216588874,VENUS GILLETTE,1527.0,Бритвы безопасные,2025-02-12 00:00:00,2025-02-12 23:59:59,65.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,21.0,Wildberries
2621,WB-Smart-Market,288465485.0,8700216588713,GILLETTE,1527.0,Бритвы безопасные,2025-02-12 00:00:00,2025-02-12 23:59:59,89.0,20.0,...,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,5.0,Wildberries
2622,WB-Smart-Market,313543155.0,7702018542048,GILLETTE,1290.0,Кассеты для бритв,2025-02-12 00:00:00,2025-02-12 23:59:59,5.0,1.0,...,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,54.0,Wildberries
2623,WB-Smart-Market,313543156.0,7702018085934,GILLETTE,1290.0,Кассеты для бритв,2025-02-12 00:00:00,2025-02-12 23:59:59,15.0,2.0,...,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,Wildberries


In [45]:
# Save the DataFrame to an Excel file
excel_file_path = 'report_data.xlsx'
combined_df.to_excel(excel_file_path, index=False)

print(f"Data saved to {excel_file_path} successfully!")

Data saved to report_data.xlsx successfully!


In [46]:
# Keep only the desired columns
columns_to_keep = ['nmID', 'vendorCode', 'brandName', 'objectID', 'objectName', 'begin', 'openCardCount', 'addToCartCount', 'ordersCount', 'ordersSumRub', 'buyoutsCount', 'buyoutsSumRub', 'cancelCount', 'cancelSumRub', 'stocksMp', 'stocksWb', 'Project', 'Marketplace']
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_df[columns_to_keep].copy()
filtered_df['begin'] = pd.to_datetime(filtered_df['begin'])
filtered_df
df_final=filtered_df.copy()

## Inserting the data

In [ ]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

Unexpected Http Driver Exception


OperationalError: Error HTTPSConnectionPool(host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net', port=8443): Read timed out. (read timeout=10) executing HTTP request attempt 1 (https://rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net:8443)

In [ ]:
# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(filtered_df.dtypes)

# Ensure the DataFrame has the correct columns
columns = ['nmID', 'vendorCode', 'brandName', 'objectID', 'objectName', 'begin', 'openCardCount', 
           'addToCartCount', 'ordersCount', 'ordersSumRub', 'buyoutsCount', 'buyoutsSumRub', 'cancelCount', 
           'cancelSumRub', 'stocksMp', 'stocksWb', 'Project', 'Marketplace']

# Reorder columns to match the expected order
data_organized = filtered_df[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in data_organized.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'order_history_wb'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data types of merged_df:
nmID                       int64
vendorCode                object
brandName                 object
objectID                   int64
objectName                object
begin             datetime64[ns]
openCardCount              int64
addToCartCount             int64
ordersCount                int64
ordersSumRub               int64
buyoutsCount               int64
buyoutsSumRub              int64
cancelCount                int64
cancelSumRub               int64
stocksMp                   int64
stocksWb                   int64
Project                   object
Marketplace               object
dtype: object
Sample data to insert: [(159488077, 'I01476', 'AND', 594, 'Массажеры электрические', Timestamp('2025-02-13 00:00:00'), 840, 64, 16, 105367, 14, 92197, 2, 13170, 0, 1, 'WB-GutenTech', 'Wildberries'), (145555249, 'SMF03PKEU', 'SMEG', 619, 'Миксеры', Timestamp('2025-02-13 00:00:00'), 23, 1, 1, 69919, 1, 69919, 0, 0, 0, 0, 'WB-GutenTech', 'Wildberries'), (252275489, '8